# <center> Ejercicios de Optimización </center>

### 1) Minimizar:

$$ x1*x4*(x1 + x2 + x3) + x3$$

#### Restricciones:
 
#### 1) $$ x1*x2*x3*x4 >= 25$$

#### 2) $$x1^2 + x3^2 + x4^2 = 40 $$

#### Límites: $$1 <= xi <= 5$$
#### Semilla inicial: $$(1, 5, 5, 1)$$

In [7]:
from scipy.optimize import minimize as mini
import numpy as np

limiteInferior = 1
limiteSuperior = 5
semilla = [1, 5, 5, 1]

def fun(x):
    funcion = x[0]* x[3] * (x[0] + x[1] + x[2]) + x[2]
    return (funcion)

def restriccion1(x):
    return (np.prod(x)- 25)

def restriccion2(x):
    return (np.sum(np.square(x)) - 40)


# Configuraciones para ejecutar algoritmo de optimización
restricciones = [{'type': 'ineq', 'fun': restriccion1},
                 {'type': 'eq', 'fun': restriccion2}]
limites = [(limiteInferior, limiteSuperior) for x in range(4)]

# Ejecución de optimización
solucion = mini(fun, semilla, method='SLSQP', bounds = limites, constraints = restricciones)

print('Solución:')
print('X1:', solucion.x[0])
print('X2:', solucion.x[1])
print('X3:', solucion.x[2])
print('X4:', solucion.x[3])
print('Funcion evaluada en la solucion', solucion.fun)

Solución:
('X1:', 1.0)
('X2:', 4.7429960656369721)
('X3:', 3.8211546642483363)
('X4:', 1.3794076394141688)
('Funcion evaluada en la solucion', 17.01401724556073)


### 2) Minimizar: $$ f = -2*x1*x2 + 2*x1 - x1^2 - 2*x2^2 $$
#### Restricciones:
#### 1) $$x1^3 - x2 = 0$$
#### 2) $$x2 - (x1 - 1)^4 - 2 >= 0$$
#### Límites $$x1: 0.5 <= x1 <= 1.5$$
#### $$x2: 1.5 <= x2 <= 2.5$$
#### Semilla inicial: $$(0, 2.5)$$

In [12]:
from scipy.optimize import minimize as mini

limitesX0 = (0.5, 1.5)
limitesX1 = (1.5, 2.5)
semilla = [0, 2.5]

def fun(x):
    funcion = 2*x[0]*x[1] + 2*x[0] - x[0]**2 - 2*(x[1]**2)
    return (-funcion)

def restriccion1(x):
    return (x[0]**3 - x[1])

def restriccion2(x):
    return (x[1] - (x[0] - 1)**4 - 2)

#%% PROGRAMA
# Configuraciones para ejecutar algoritmo de optimización
restricciones = [{'type': 'eq', 'fun': restriccion1},
                 {'type': 'ineq', 'fun': restriccion2}]
limites = (limitesX0, limitesX1)

# Ejecución de optimización
solucion = mini(fun, semilla, method='SLSQP', bounds = limites, constraints = restricciones)

print('Solución:')
print('X1', solucion.x[0])
print('X2', solucion.x[1])
print('Funcion evaluada en la solucion:', solucion.fun)


Solución:
('X1', 1.2608931426383998)
('X2', 2.0046328757906529)
('Funcion evaluada en la solucion:', 2.0499154720932884)


### 3) Maximizar: $$f = x + y$$
#### Restricciones:
#### 1) $$50x + 24y <= 2400$$
#### 2) $$30x + 33y <= 2100$$
#### Límites: $$x, y <= 0$$

In [13]:
import numpy as np
from scipy.optimize import linprog as lp

# Coeficientes de Función
cFuncionMin = [1, 1] # 1x + 1y

# Coeficientes de Restricciones
aRestricciones = [[50, 24], [30, 33]] # [50x + 24y], [30x + 33y]
bRestricciones = [2400, 2100]

# Límites
limitesX = (0, None)
limitesY = (0, None)

cFuncionMax = -np.array(cFuncionMin) # Maximización en lugar de minimización
limites = (limitesX, limitesY) 

solucion = lp(cFuncionMax, A_ub = aRestricciones, b_ub = bRestricciones, bounds = limites)
funcionReal = - solucion.fun # Ocurrió una maximización en lugar de una minimización


print('Solución:')
print('X:',solucion.x[0])
print('Y:', solucion.x[0])
print('Funcion evaluada en la solucion:', funcionReal)

Solución:
('X:', 30.967741935483872)
('Y:', 30.967741935483872)
('Funcion evaluada en la solucion:', 66.451612903225808)


### Maximizar ingreso: $$\sum (p(t) * x(t))$$

#### $$p(t):$$ Precio del producto en el intervalo de tiempo 't'
#### $$x(t):$$ Cantidad de producto vendido (demanda) en el intervalo de tiempo 't'

### Restricciones:

#### 1) El total de productos vendidos no puede superar el inventario total 
$$(s0) \sum (x(t)) <= s0$$

#### 2) 
$$x(t) >= 0$$ (cantidad de productos no negativa)
#### 3) 
$$x(t) <= \frac{A}{B}$$ (valor según modelo para que precios no sean negativos)

#### Modelo: Precio en función del tiempo y la demanda
#### $$p(x, t) = (A - B * x(t)) * \frac{D}{D+1}$$

A,B y D son parámetros que resultan de la caracterización del comportamiento
Del ingreso como producto del precio por la demanda, a lo largo del tiempo.
Puede darles el valor que considere conveniente.

In [16]:
# Limitación de número de decimales
np.set_printoptions(precision = 2, suppress = True)

tiempoTotal = 10 # Total de Horas
factorEscala = 10000 # Factor para que el resultado sea de orden 1, facilitando convergencia
A = 5000 # Precio máximo
B = 100 # Rebaja en precio según la demanda
D = 20 # Término para disminución del precio a medida que avanza el tiempo
s0 = 200 # Inventario total

# Implementación de modelo de precios en función del tiempo y la demanda
def precios(x):
    # Variables globales a utilizar en modelo
    global A, B, D, arregloTiempo
    
    # Inicializar arreglo precios
    listaPrecios = []
    
    # Ciclo para calcular precio en cada intervalo de tiempo
    for (indice, tiempo) in enumerate(arregloTiempo):
        precio = (A - B * x[indice]) * (D / (D + tiempo))
        # Agregar precio en arreglo sólo si no es negativo
        if (precio >= 0): listaPrecios.append(precio)
        else: listaPrecios.append(0)
    
    # Retornar lista de precios como arreglo NumPy
    return (np.array(listaPrecios))

# Función a optimizar: Ingreso
def ingreso(x):
    producto = np.dot(precios(x), x) # Ingresos: Precio * Demanda
    return (- producto / factorEscala) # Maximizar en lugar de minimizar

# Restricciones
def restriccionInventario(x):
    global s0
    return (s0 - np.sum(x))


arregloTiempo = np.arange(1, tiempoTotal + 1) # N° Hora

# Arreglo Semilla de demanda (Aleatorio: Entre 0 y promedio)
promedioDemanda = (s0 // tiempoTotal)
x0 = np.random.rand(tiempoTotal) * promedioDemanda
xPromedio = np.full(tiempoTotal, promedioDemanda)

# Configuraciones para ejecutar algoritmo de optimización
# Restricciones
restricciones = [{'type': 'ineq', 'fun': restriccionInventario}]
# Límites
limiteInferior = 0
limiteSuperior = A / B
limites = [(limiteInferior, limiteSuperior) for i in range(tiempoTotal)]

# Ejecución de optimización
soluciones = mini(ingreso, x0, method='SLSQP', bounds = limites, constraints = restricciones)
ingresoTotal = - ingreso(soluciones.x) * factorEscala

# Arreglo de tiempo (N° Hora)
print('Arreglo de Tiempo:')
print(arregloTiempo)
print('')

# Arreglo semilla de demanda:
print('Arreglo Semilla de Demanda:')
print(x0)
print('Suma Arreglo Semilla Demanda', np.sum(x0))
print('')

# Resumen solución
# Arreglo de demanda de Productos por hora
print('Demanda Promedio de Productos (por Hora): ')
print(xPromedio)
print('Demanda Óptima de Productos por Hora: ')
print(soluciones.x)
print('')

# Arreglo de precio de productos por hora
print('Precio Estimado de Productos por Hora (para Demanda Óptima):')
print(precios(soluciones.x))
print('')

# Ingreso total
print('Ingreso total: $', ingresoTotal)

Arreglo de Tiempo:
[ 1  2  3  4  5  6  7  8  9 10]

Arreglo Semilla de Demanda:
[ 18.29  18.19   6.84   6.17  13.74  12.65  19.62   5.72  10.27   2.55]
('Suma Arreglo Semilla Demanda', 114.00663246984689)

Demanda Promedio de Productos (por Hora): 
[ 20.  20.  20.  20.  20.  20.  20.  20.  20.  20.]
Demanda Óptima de Productos por Hora: 
[ 18.29  18.19   6.84   6.17  13.74  12.65  19.62   5.72  10.27   2.55]

Precio Estimado de Productos por Hora (para Demanda Óptima):
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

('Ingreso total: $', 0.0)
